In [1]:
# coding: utf-8
"""
ノード分割アルゴリズム部分のコードの書き換え
alpha = 全単語数に対する語彙の頻度数, beta = 頻度の割合
"""
from igraph import *
import csv
import collections
import pickle
import numpy as np
from openopt import QP
import copy
import re

# csvファイルの読み込み
def readcsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f)
    arr = [row for row in dataReader]
    return arr

def writecsv(arr, path):
    f = open(path, "ab")
    dataWriter = csv.writer(f)
    dataWriter.writerows(arr)
    f.close()

def readdump(path):
    f = open(path, "r")
    arr = pickle.load(f)
    f.close()
    return arr

# 有向エッジリストを入力して、重み付き無向ネットワークを出力する
def cal_edgelist_to_network(list_edge):
    # 有向エッジリストを無向エッジリストに変換する
    list_edge = [tuple(sorted(row)) for row in list_edge]
    # ノードリスト
    list_vertices = list(set([word for row in list_edge for word in row]))
    # エッジリストとそのweightを作成
    tuple_edge, tuple_weight = zip(*collections.Counter(list_edge).items())
    return {"vertex": list_vertices, "edge": list(tuple_edge), "weight": list(tuple_weight)}
    
    
# クラスタリング済みのネットワークを元にサブグラフのリストを作成
# vertexには全てのvertexを代入する（PageRankを計算するため）
def cal_cluster_to_network(dict_network):
    if dict_network.has_key("cluster") == False:
        print "クラスタリングができていません"
    
    # クラスタごとにwordをまとめる
    dict_cluster = collections.defaultdict(list)
    for word, cluster in zip(dict_network["vertex"], dict_network["cluster"]):
        dict_cluster[cluster].append(word)

    # リストに変換
    list_cluster_vertex = [row[1] for row in dict_cluster.items()]
    
    # 同様にエッジとウェイトのリストも作成する
    list_cluster_edge = []
    list_cluster_weight = []
    for cluster_vertex in list_cluster_vertex:
        list_cluster_edge_one = []
        list_cluster_weight_one = []
        # エッジリストの中に、一つでもノードが含まれていれば、そのクラスのノードに含める
        for row, weight in zip(dict_network["edge"], dict_network["weight"]):
            # and と or を切り替えることによって性能の比較
            if row[0] in cluster_vertex or row[1] in cluster_vertex:
                list_cluster_edge_one.append(row)
                list_cluster_weight_one.append(weight)
        list_cluster_edge.append(list_cluster_edge_one)
        list_cluster_weight.append(list_cluster_weight_one)
    
    # まとめる
    list_dict_network = [{"vertex": dict_network["vertex"],
                          "edge": cluster_edge,
                          "weight": cluster_weight}
                         for cluster_edge, cluster_weight
                         in zip(list_cluster_edge, list_cluster_weight)]
    
    return list_dict_network

# f_measureを計算する
def cal_f_measure(list_predict_measure):
    # 生成したクラスタ内のカウント
    dict_predict_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_predict_cluster[row[0]].append(row[1])
        
    # もとあるクラス内のカウント
    dict_measure_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_measure_cluster[row[1]].append(row[0])
    
    # local_purityの計算
    list_purity = []
    for row in dict_predict_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_purity.append([major_class, class_num])
    purity = float(np.sum(zip(*list_purity)[0])) / np.sum(zip(*list_purity)[1])
    print "Purity: ", purity
    
    # inverse_purityの計算
    list_inverse_purity = []
    for row in dict_measure_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_inverse_purity.append([major_class, class_num])
    inverse_purity = float(np.sum(zip(*list_inverse_purity)[0])) / np.sum(zip(*list_inverse_purity)[1])
    print "Inverse Purity: ", inverse_purity
    
    print "F-value: ", 2 / (1 / purity + 1 / inverse_purity)
    
# 凸２次計画問題を解いてp(topic)を求めるための関数
def cal_prob_topic(dict_network_master, list_dict_network_sub):
    prob_master = np.array([row[1] for row in sorted(zip(dict_network_master["vertex"], dict_network_master["page_rank"]), key=lambda x: x[0])])
    
    for i, dict_network_sub in enumerate(list_dict_network_sub):
        if i == 0:
            prob_sub = np.array([row[1] for row in sorted(zip(dict_network_sub["vertex"], dict_network_sub["page_rank"]), key=lambda x: x[0])])
        else:
            list_tmp = np.array([row[1] for row in sorted(zip(dict_network_sub["vertex"], dict_network_sub["page_rank"]), key=lambda x: x[0])])
            prob_sub = np.vstack((prob_sub, list_tmp))
    
    H = 2 * prob_sub.dot(prob_sub.T)
    f = -2 * prob_master.dot(prob_sub.T)
    Aeq = np.ones(len(list_dict_network_sub))
    beq = 1
    lb = np.zeros(len(list_dict_network_sub))
    
    p = QP(H, f, Aeq=Aeq, beq=beq, lb=lb)
    r = p.solve("cvxopt_qp")
    k_opt = r.xf
    return k_opt

In [2]:
# クラスタリング後に、ノードごとにどのクラスタにエッジを持っているのか計算
def cal_cluster_to_edge(dict_network, low_fleq=0.05, low_rate=0.8, flag=0):
    if dict_network.has_key("cluster") == False:
        print "クラスタリングができていません"
    
    # wordをclusterに変換するための辞書を作成する
    dict_word_to_cluster = {}
    for word, cluster in zip(dict_network["vertex"], dict_network["cluster"]):
        dict_word_to_cluster[word] = cluster
        
    # wordをidに変換するための辞書を作成する
    dict_word_to_id = {}
    for i, word in enumerate(dict_network["vertex"]):
        dict_word_to_id[word] = i
    
    # clusterの数
    num_cluster = len(set(dict_network["cluster"]))
    
    # id_to_clusterのマトリックス
    matrix_id_to_cluster = np.zeros((len(dict_word_to_id), num_cluster))
    for row, weight in zip(dict_network["edge"], dict_network["weight"]):
        matrix_id_to_cluster[dict_word_to_id[row[0]]][dict_word_to_cluster[row[1]]] += weight
        matrix_id_to_cluster[dict_word_to_id[row[1]]][dict_word_to_cluster[row[0]]] += weight
        
    #総単語数を求める
    total_voc = np.sum(matrix_id_to_cluster)
    
    # 指定したハイパーパラメータよりも高い値を記録した単語を所属クラスタとともに辞書に登録する
    dict_word_to_list_cluster = {}
    for cluster, word, row in zip(dict_network["cluster"], dict_network["vertex"], matrix_id_to_cluster):
        top_num = max(row)
        if float(np.sum(row))/total_voc>=low_fleq and len(np.where(row/top_num>=low_rate)[0])>=2:
            print word
            dict_word_to_list_cluster[word] = np.where(row/top_num>=low_rate)[0]
            
    # 分割する単語が存在するかしないかflagを立てる
    if len(dict_word_to_list_cluster) > 0:
        outflag = True
    else:
        outflag = False

    # 新しく分裂するノードクラスターを元のクラスターに記録
    for word, row in dict_word_to_list_cluster.items():
        for num in row:
            # dict_word_to_cluster[word+"_"+str(num)] = num
            dict_word_to_cluster[word+"_"+str(num)] = dict_word_to_cluster[word]
    
    # ここの計算が間違ってそう => diffsplitで確認したが大丈夫そう・・・
    for word in dict_word_to_list_cluster.keys():
        list_edge_new = []
        list_weight_new = []
        for row, weight in zip(dict_network["edge"], dict_network["weight"]):
            if row[0] == word:
                if dict_word_to_cluster[row[1]] in dict_word_to_list_cluster[word]:
                    list_edge_new.append([row[0]+"_"+str(dict_word_to_cluster[row[1]]), row[1]])
                    list_weight_new.append(weight)
                else:
                    for num in dict_word_to_list_cluster[word]:
                        # flagの値によって、weightの計算法を切り替え
                        if flag == 0:
                            weight_tmp = float(weight)/len(dict_word_to_list_cluster[word])
                            list_edge_new.append([row[0]+"_"+str(num), row[1]])
                            list_weight_new.append(weight_tmp)
                        else:
                            weight_tmp = weight/len(dict_word_to_list_cluster[word])
                            if weight_tmp > 0:
                                list_edge_new.append([row[0]+"_"+str(num), row[1]])
                                list_weight_new.append(weight_tmp)
                                
            elif row[1] == word:
                if dict_word_to_cluster[row[0]] in dict_word_to_list_cluster[word]:
                    list_edge_new.append([row[0], row[1]+"_"+str(dict_word_to_cluster[row[0]])])
                    list_weight_new.append(weight)
                else:
                    for num in dict_word_to_list_cluster[word]:
                        # flagの値によって、weightの計算法を切り替え
                        if flag == 0:
                            weight_tmp = float(weight)/len(dict_word_to_list_cluster[word])
                            list_edge_new.append([row[0], row[1]+"_"+str(num)])
                            list_weight_new.append(weight_tmp)
                        else:
                            weight_tmp = weight/len(dict_word_to_list_cluster[word])
                            if weight_tmp > 0:
                                list_edge_new.append([row[0], row[1]+"_"+str(num)])
                                list_weight_new.append(weight_tmp)
            else:
                list_edge_new.append([row[0], row[1]])
                list_weight_new.append(weight)
        else:
            dict_network["edge"] = copy.deepcopy(list_edge_new)
            dict_network["weight"] = copy.deepcopy(list_weight_new)
            list_vertices = list(set([word for row in dict_network["edge"] for word in row]))
            dict_network["vertex"] = list_vertices
    
    return dict_network, outflag

###メイン部分
1. 有向エッジリストから無向重み付きエッジリストを作成
2. louvain法によるクラスタリング
3. クラスタごとにpagerankを計算し、p(word|topic)とする
4. クラスタごとにp(word|topic)を出力する辞書を作成

In [24]:
# エッジリストの読み込み
list_edge = readcsv("./files/rakuten_corpus/rakuten_corpus_edgelist.csv")
# 元のネットワークを作成する（無向）
dict_network_master = cal_edgelist_to_network(list_edge)
# g = Graph(directed=True)
g_master = Graph()
g_master.add_vertices(dict_network_master["vertex"])
g_master.add_edges(dict_network_master["edge"])
# 元のネットワークのpagerankを求める
dict_network_master["page_rank"] = g_master.pagerank(directed=False, weights=dict_network_master["weight"])

flag=True
counter=0
while flag:
    # louvain法によるクラスタリング、vertexと同じ長さのクラスタ番号が書かれたリストがreturn
    dict_network_master["cluster"] = g_master.community_fastgreedy(weights=dict_network_master["weight"]).as_clustering(n=6).membership
    # クラスタリングの結果をもとに、どのノードから何本のエッジが出ているか計算する
    dict_network_master, flag = cal_cluster_to_edge(dict_network_master, low_fleq=0.002, low_rate=0.6, flag=0)
    counter +=1
    print counter, "回目"
    print len(dict_network_master["weight"])
    print len(dict_network_master["edge"])
    g_master = Graph()
    g_master.add_vertices(dict_network_master["vertex"])
    g_master.add_edges(dict_network_master["edge"])

dict_network_master["cluster"] = g_master.community_fastgreedy(weights=dict_network_master["weight"]).as_clustering(n=6).membership
# 元のネットワークのpagerankを求める
dict_network_master["page_rank"] = g_master.pagerank(directed=False, weights=dict_network_master["weight"])
# クラスタ結果をもとにサブグラフのリストを作成
list_dict_network_sub = cal_cluster_to_network(dict_network_master)
# サブクラスタごとに中心性の計算
for i, dict_network_sub in enumerate(list_dict_network_sub):
    g_sub = Graph()
    g_sub.add_vertices(dict_network_sub["vertex"])
    g_sub.add_edges(dict_network_sub["edge"])
    list_dict_network_sub[i]["page_rank"] = g_sub.pagerank(directed=False, weights=dict_network_sub["weight"])
print "クラスタ数: ", len(list_dict_network_sub)

# トピックごとにwordを入力したらp(word|topic)が出るような辞書を作成
list_dict_prob = []
for i in range(len(list_dict_network_sub)):
    list_word_page = sorted(zip(list_dict_network_sub[i]["vertex"], list_dict_network_sub[i]["page_rank"]), key=lambda x: x[1], reverse=True)
    list_word_page_rev = []
    for row in list_word_page:
        pattern = "_[0-9]+"
        word = re.sub(pattern, "", row[0])
        list_word_page_rev.append([word, row[1]]) 
    list_dict_prob.append({row[0]: row[1] for row in list_word_page_rev})
    
# 凸２次計画問題を解いて、p(topic)を求める
list_prob_topic = cal_prob_topic(dict_network_master, list_dict_network_sub)

温泉
プラン
機会
あと
なかった
ビジネスホテル
値段
いい
十分
得
他
仕事
よい
充実
欲しい
必要
内容
大変
家族
シャワー
きれい
お世話になりました
お願い
気持ちよい
満足
宿
用意
残念
掃除
無い
1 回目
10914
10914
ない
2 回目
11124
11124
3 回目
11124
11124
クラスタ数:  6

------------------------- OpenOpt 0.5625 -------------------------
problem: unnamed   type: QP
solver: cvxopt_qp
     pcost       dcost       gap    pres   dres
 0: -2.5309e-03 -1.0031e+00  1e+00  2e-16  3e+00
 1: -2.5313e-03 -1.3055e-02  1e-02  1e-16  3e-02
 2: -2.5580e-03 -3.0008e-03  4e-04  7e-17  1e-03
 3: -2.6018e-03 -2.6361e-03  3e-05  2e-16  2e-05
 4: -2.6033e-03 -2.6041e-03  8e-07  2e-16  2e-07
 5: -2.6033e-03 -2.6033e-03  8e-09  9e-17  2e-09
Optimal solution found.
istop: 1000 (optimal)
Solver:   Time Elapsed = 0.0 	CPU Time Elapsed = 0.0
objFuncValue: -0.0026032717 (feasible, MaxResidual = 0)


In [38]:
num = 4
list_tmp = sorted(list_dict_prob[num].items(), key=lambda x: x[1], reverse=True)
for i in range(10):
    print list_tmp[i][0], list_tmp[i][1]

予約 0.0336970436092
出張 0.0208632894504
確認 0.00932700105509
ツイン 0.009225442074
是非 0.00846394105412
連絡 0.00842891556384
泊 0.00830725519894
変更 0.00806830354263
改善 0.00784465830907
お世話 0.00783298015713


### 定量評価部分
1. 形態素解析済みセンテンスを読み込み、確率を計算し、どのクラスに属するか計算
2. 予測クラスと実際クラスのリストを作成する

In [27]:
list_dict_words = dict_network_master["vertex"]
list_sep_words = readdump("./files/rakuten_corpus/annotation/all_sep.dump")
list_sep_words_rev = []
for row in list_sep_words:
    list_sep_words_rev.append([row[2], row[1]])
# 確率が最大になるクラスを予想する
# 実質ラベルがintじゃない場合は、エラーとして、記録しない
list_predict_measure = []
list_words_rev = []
error_count = 0
for row in list_sep_words_rev:
    try:
        class_tmp = 0
        prob_tmp = 0
        predict_class = []
        for num, dict_prob in enumerate(list_dict_prob):
            prob_tmp_tmp = 1
            for word in row[0]:
                prob_tmp_tmp *= float(dict_prob[word])
            prob_tmp_tmp *= list_prob_topic[num]
            if prob_tmp_tmp > prob_tmp:
                class_tmp = num
                prob_tmp = prob_tmp_tmp
        list_predict_measure.append([class_tmp, row[1]])
    except:
        error_count += 1
print "エラーデータ数: ", error_count

エラーデータ数:  2


###計算結果の表示

* 0.01, 0.6

In [12]:
cal_f_measure(list_predict_measure)

Purity:  0.592592592593
Inverse Purity:  0.651651651652
F-value:  0.620720479128


* 0.001, 0.6

In [15]:
cal_f_measure(list_predict_measure)

Purity:  0.593093093093
Inverse Purity:  0.670670670671
F-value:  0.629500787917


* 0.002, 0.6

In [18]:
cal_f_measure(list_predict_measure)

Purity:  0.595595595596
Inverse Purity:  0.679179179179
F-value:  0.634647211012


* 0.003, 0.6

In [21]:
cal_f_measure(list_predict_measure)

Purity:  0.601601601602
Inverse Purity:  0.672172172172
F-value:  0.63493198503


* 0.004, 0.6

In [24]:
cal_f_measure(list_predict_measure)

Purity:  0.592592592593
Inverse Purity:  0.666166166166
F-value:  0.627229217289


* 0.005, 0.6

In [28]:
cal_f_measure(list_predict_measure)

Purity:  0.594094094094
Inverse Purity:  0.666166166166
F-value:  0.628069292485


*

In [52]:
cal_f_measure(list_predict_measure)

Purity:  0.589089089089
Inverse Purity:  0.657157157157
F-value:  0.621264236124


* 100, 0.6

In [55]:
cal_f_measure(list_predict_measure)

Purity:  0.592592592593
Inverse Purity:  0.666166166166
F-value:  0.627229217289


* 50, 0.6

In [58]:
cal_f_measure(list_predict_measure)

Purity:  0.602102102102
Inverse Purity:  0.67017017017
F-value:  0.634315275149


* 100, 0.7, or、切り捨てで計算した場合

In [23]:
cal_f_measure(list_predict_measure)

Purity:  0.567091087169
Inverse Purity:  0.629285014691
F-value:  0.596571467059


* 100, 0.9, or, 切り捨てで計算した場合

In [215]:
cal_f_measure(list_predict_measure)

Purity:  0.487093835408
Inverse Purity:  0.623443668387
F-value:  0.546898356081


* 50, 0.8, or, 切り捨てで計算した結果

In [233]:
cal_f_measure(list_predict_measure)

Purity:  0.494078348011
Inverse Purity:  0.638931065897
F-value:  0.557245159054


* 50, 0.7, or, 切り捨てで計算した結果

In [237]:
cal_f_measure(list_predict_measure)

Purity:  0.486486486486
Inverse Purity:  0.649559672032
F-value:  0.556318949262


In [18]:
cal_f_measure(list_predict_measure)

Purity:  0.492256301245
Inverse Purity:  0.639234740358
F-value:  0.556199417134
